## Get the Data Source, and Prepare Training and Test set

In [26]:
#import all the required package
import pandas as pd

In [27]:
#import all training dataset
df_620 = pd.read_csv('./scraping_data/result_taobao.csv', header=None)
df_621 = pd.read_csv('./scraping_data/result_621_2.csv', header=None)
df_622_1 = pd.read_csv('./scraping_data/result_622.csv', header=None)
df_622_2 = pd.read_csv('./scraping_data/result_622_add.csv', header=None)
df_622 = pd.concat([df_622_1,df_622_2])
df_623 = pd.read_csv('./scraping_data/result_smartphone_623.csv', header=None)
df_624 = pd.read_csv('./scraping_data/result_ereader_624.csv', header=None)
df_625 = pd.read_csv('./scraping_data/result_625.csv', header=None)

pd.set_option('display.max_colwidth', -1) #set this for remove truncated by jupyter/python

In [13]:
print('df_620',df_620.head(1))
print('df_621',df_621.head(1))
print('df_622',df_622.head(1))
print('df_623',df_623.head(1))
print('df_624',df_624.head(1))
print('df_625',df_625.head(1))

df_620                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [14]:
df_620[0][1:2].to_string()

"1    summer icy sleeves sun protection gloves for men and women anti-uv thin long ice silk sun protection sleeve sleeves driving arm sleeve black thumb models product details of summer icy sleeves sun protection gloves for men and women anti-uv thin long ice silk sun protection sleeve sleeves driving arm sleeve black thumb models <img src='https:ororimg.alicdn.comorimgextraori3or3406874323ortb2iopbhuuil1jjszfrxxb3xfxa_!!3406874323.png'> <img id='desc-module-1' src='https:ororassets.alicdn.comorkissyor1.0.0orbuildorimglazyloadorspaceball.gif'> <img src='https:ororimg.alicdn.comorimgextraori2or929593080ortb2npmzd2nw1ejjszfqxxa8svxa_!!929593080.jpg' alt=''> \xa0 <img src='https:ororimg.alicdn.comorimgextraori4or929593080ortb2orzkgh3il1jjszpfxxcruvxa_!!929593080.jpg' align='absmiddle'> <img align='absmiddle' src='https:ororimg.alicdn.comorimgextraori2or929593080ortb2rzvlp9fjpufjsszhxxabuvxa_!!929593080.jpg'> <img align='absmiddle' src='https:ororimg.alicdn.comorimgextraori2or929593080ortb

In [28]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag, FreqDist
import random
import pickle

In [37]:
# read the labelled all the source data
def DataSources():
    data_620 = df_620[0].to_string()
    data_621 = df_621[0].to_string()
    data_622 = df_622[0].to_string()
    data_623 = df_623[0].to_string()
    data_624 = df_624[0].to_string()
    data_625 = df_625[0].to_string()
    return data_620, data_621, data_622, data_623, data_624,data_625

In [30]:
# prepare the data
def PrepareData():
    train_620, train_621, train_622, train_623, train_624, train_625 = DataSources()
    documents = []
    all_words = []
    
#    j is adjective, r is adverb, and v is verb
    allowed_word_types = ["J","R","V"]
#     allowed_word_types = ["J"]

    for p in train_620.split('\n'):
        documents.append((p,"Others"))
        words = word_tokenize(p)
        pos = pos_tag(words)
        for w in pos:
            if w[1][0] in allowed_word_types:
                all_words.append(w[0].lower())


    for p in train_621.split('\n'):
        documents.append((p,"Consumer Electronics > Cameras and Camcorders"))
        words = word_tokenize(p)
        pos = pos_tag(words)
        for w in pos:
            if w[1][0] in allowed_word_types:
                all_words.append(w[0].lower())

    for p in train_622.split('\n'):
        documents.append((p,"Consumer Electronics > Home Entertainment Systems"))
        words = word_tokenize(p)
        pos = pos_tag(words)
        for w in pos:
            if w[1][0] in allowed_word_types:
                all_words.append(w[0].lower())

    for p in train_623.split('\n'):
        documents.append((p,"Technology & Computing > Consumer Electronics > Smartphones"))
        words = word_tokenize(p)
        pos = pos_tag(words)
        for w in pos:
            if w[1][0] in allowed_word_types:
                all_words.append(w[0].lower())

    for p in train_624.split('\n'):
        documents.append((p,"Technology & Computing > Consumer Electronics > Tablets and E-readers"))
        words = word_tokenize(p)
        pos = pos_tag(words)
        for w in pos:
            if w[1][0] in allowed_word_types:
                all_words.append(w[0].lower())
                
    for p in train_625.split('\n'):
        documents.append((p,"Technology & Computing > Consumer Electronics > Wearable Technology"))
        words = word_tokenize(p)
        pos = pos_tag(words)
        for w in pos:
            if w[1][0] in allowed_word_types:
                all_words.append(w[0].lower())
                
    save_documents = open("saved/documents.p", "wb")
    pickle.dump(documents, save_documents)
    save_documents.close()

    all_words = FreqDist(all_words)
#     word_features = list(all_words.keys())[:5000]
    word_features = list(all_words.keys())[:]

    save_word_features = open("saved/word_features5k.p", "wb")
    pickle.dump(word_features, save_word_features)
    save_word_features.close()

    features = [(find_features(rev, word_features), category) for (rev, category) in documents]
    return features


def find_features(document, word_features):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features


def TestTrainData():
    featuresets = PrepareData()
    random.shuffle(featuresets)
#    print(len(featuresets))
    testing_set = featuresets[10000:]
    training_set = featuresets[:10000]
    return training_set, testing_set

## Train Classifiers and store them

In [31]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from nltk import NaiveBayesClassifier, classify
import pickle


def TrainClassifiers():
    training_set, testing_set = TestTrainData()

    classifiers = list()
    classifier_name = list()

    NaiveBayesClassifier_classifier = NaiveBayesClassifier.train(training_set)
    classifiers.append(NaiveBayesClassifier_classifier)
    classifier_name.append("NaiveBayesClassifier")

    MNB_classifier = SklearnClassifier(MultinomialNB())
    MNB_classifier.train(training_set)
    classifiers.append(MNB_classifier)
    classifier_name.append("MultinomialNBClassifier")

    BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
    BernoulliNB_classifier.train(training_set)
    classifiers.append(BernoulliNB_classifier)
    classifier_name.append("BernoulliNBClassifier")

    LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
    LogisticRegression_classifier.train(training_set)
    classifiers.append(LogisticRegression_classifier)
    classifier_name.append("LogisticRegressionClassifier")

    LinearSVC_classifier = SklearnClassifier(LinearSVC())
    LinearSVC_classifier.train(training_set)
    classifiers.append(LogisticRegression_classifier)
    classifier_name.append("LinearSVCClassifier")

    SGDC_classifier = SklearnClassifier(SGDClassifier())
    SGDC_classifier.train(training_set)
    classifiers.append(SGDC_classifier)
    classifier_name.append("SGDClassifier")

    SaveClassifiers(classifiers, classifier_name)

    return classifiers


def SaveClassifiers(classifiers, classifier_name):

    for i in range(0, len(classifiers)):
        save_classifier_path = open("saved/" + classifier_name[i] + ".p", "wb")
        pickle.dump(classifiers[i], save_classifier_path)
        save_classifier_path.close()

    save_classifier_path = open("saved/classifier_name.p", "wb")
    pickle.dump(classifier_name, save_classifier_path)

## Load the trained classifiers

In [32]:
import random
import pickle
from nltk.tokenize import word_tokenize
from nltk import classify


def find_features(document, word_features):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features


def LoadData(shuffle=False):

    documents_f = open("saved/documents.p", "rb")
    documents = pickle.load(documents_f)
    documents_f.close()

    documents_f = open("saved/word_features5k.p", "rb")
    word_features = pickle.load(documents_f)
    documents_f.close()

    features = [(find_features(rev, word_features), category) for (rev, category) in documents]

    if shuffle:
        random.shuffle(features)

    testing_set = features[100:]
    training_set = features[:100]

    return training_set, testing_set


def LoadClassifiers():
    document = open("saved/classifier_name.p", "rb")
    classifier_name = pickle.load(document)
    document.close()
    # print(classifier_name)
    classifiers = list()

#     training_set, testing_set = LoadData()

    for name in classifier_name:
        document = open("saved/" + name + ".p", "rb")
        classifier = pickle.load(document)
        classifiers.append(classifier)
        document.close()

    return classifiers


def LoadFeatures():
    documents_f = open("saved/word_features5k.p", "rb")
    word_features = pickle.load(documents_f)
    documents_f.close()
    return word_features

## Classifier Confidence

In [33]:
from nltk.classify import ClassifierI
from statistics import mode, StatisticsError


class VoteClassifier(ClassifierI):
    def __init__(self, classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
#         ret = "neg"
        
        try:
            ret = mode(votes)
        except StatisticsError:
            print("Caught1")
            pass
        return ret

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        try:
            choice_votes = votes.count(mode(votes))
            conf = choice_votes / len(votes)
            return conf
        except StatisticsError:
            print("Caught2")
            return 0.5

## Category Classifier : Which categories?

In [34]:
class IABClassification:

    def __init__(self):
        classifiers = TrainClassifiers()
#         classifiers = LoadClassifiers()
#         self.votedClassifier = VoteClassifier(classifiers)
#         self.new_features = LoadFeatures()

    def Analyse(self, text):
        new_features = find_features(text, self.new_features)
        return self.votedClassifier.classify(new_features), self.votedClassifier.confidence(new_features)
#         return self.votedClassifier.classify(new_features), self.votedClassifier.confidence(new_features), text

## Creating classifier for IABClassification

In [35]:
# turn off pickle warning
import warnings
warnings.filterwarnings('ignore')

In [38]:
s = IABClassification()